In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.models import Model

In [2]:
def read_data(path):
  data = pd.read_csv(path)
  features = data.iloc[:,:-1].values
  labels = data.iloc[:,-1] - 1
  return features, labels


In [3]:
path = 'https://raw.githubusercontent.com/UMBInal/data/main/data2.csv'

x, y = read_data(path)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size= 0.1, random_state=42)

In [6]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1],1))
x_val = x_val.reshape((x_val.shape[0], x_val.shape[1],1))

In [7]:
n_classes = len(np.unique(y))
n_classes

5

In [8]:
class CNN(Model):

  def __init__(self, n_neurons, n_classes, dropout):
    super(CNN, self).__init__()
    self.n_neurons = n_neurons
    self.n_classes = n_classes
    self.dropout = dropout

    self.conv1 = keras.layers.Conv1D(filters = 32, kernel_size = 2, input_shape = x_train[1:], use_bias = True, activation = 'relu')
    self.maxpool1 = keras.layers.MaxPool1D(pool_size=2)
    self.dense1 = keras.layers.Dense(n_neurons, activation = 'relu')
    self.dropout1 = keras.layers.Dropout(dropout)
    self.dense2 = keras.layers.Dense(n_neurons, activation = 'relu')
    self.dropout2 = keras.layers.Dropout(dropout)
    self.softmax = keras.layers.Dense(n_classes, activation = 'softmax')

  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.maxpool1(x)
    x = tf.keras.layers.Flatten()(x)
    x = self.dense1(x)
    x = self.dropout1(x)
    x = self.dense2(x)
    x = self.dropout2(x)
    return self.softmax(x)

In [9]:
model = CNN(64, n_classes, 0.10)

In [10]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [11]:
# Enable eager execution for debugging
tf.config.run_functions_eagerly(True)

In [12]:
model.fit(x_train, y_train, epochs = 10, batch_size= 32, validation_data=(x_val, y_val))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


826/826 [==============================] - 44s 53ms/step - loss: 0.3495 - accuracy: 0.8771 - val_loss: 0.2457 - val_accuracy: 0.9052
Epoch 2/10
826/826 [==============================] - 44s 53ms/step - loss: 0.2443 - accuracy: 0.9039 - val_loss: 0.2240 - val_accuracy: 0.9127
Epoch 3/10
826/826 [==============================] - 42s 50ms/step - loss: 0.2218 - accuracy: 0.9143 - val_loss: 0.2146 - val_accuracy: 0.9100
Epoch 4/10
826/826 [==============================] - 42s 51ms/step - loss: 0.2139 - accuracy: 0.9181 - val_loss: 0.2112 - val_accuracy: 0.9093
Epoch 5/10
826/826 [==============================] - 42s 50ms/step - loss: 0.2045 - accuracy: 0.9221 - val_loss: 0.1980 - val_accuracy: 0.9206
Epoch 6/10
826/826 [==============================] - 40s 49ms/step - loss: 0.1941 - accuracy: 0.9259 - val_loss: 0.1934 - val_accuracy: 0.9196
Epoch 7/10
826/826 [==============================] - 43s 52ms/step - loss: 0.1900 - accuracy: 0.9269 - val_loss: 0.1952 - val_accuracy: 0.9134
Epo

In [17]:
test_loss, test_accuracy = model.evaluate(x_test,y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

230/230 [==============================] - 5s 23ms/step - loss: 0.1816 - accuracy: 0.9283
Test loss: 0.18156562745571136, Test accuracy: 0.9282792210578918
